In [29]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [30]:
model = SentenceTransformer("emilyalsentzer/Bio_ClinicalBERT")

Downloading:   0%|          | 0.00/391 [00:04<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.61M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/23.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

No sentence-transformers model found with name /home/ricardo/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/ricardo/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertFo

In [32]:
embeddings = model.encode("Patient presents to the ER with no adduction of dexter oculus", show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
embeddings

array([ 1.43734232e-01, -2.47471109e-01, -1.05535328e-01, -1.42186433e-01,
        4.32358295e-01, -1.21277474e-01,  1.45889536e-01,  9.81251895e-02,
       -3.80362898e-01, -8.11375603e-02,  1.63590107e-02,  2.73184448e-01,
        1.46680295e-01, -3.32147665e-02, -3.92429605e-02,  1.45066589e-01,
        2.84236580e-01, -2.43330061e-01, -2.14798614e-01,  3.79987657e-01,
       -7.70591479e-03, -1.06782475e-02, -2.52945006e-01, -4.26073521e-01,
        2.45417461e-01, -3.58902842e-01, -1.26812458e-01,  6.37156427e-01,
        9.68703032e-02,  2.84381807e-01, -5.73599711e-02,  5.57643473e-02,
       -1.03775961e-02, -1.05977982e-01, -8.02437961e-02,  2.32724786e-01,
        4.55117732e-01,  4.13280010e-01, -9.34125390e-03,  4.33911830e-01,
        4.83619571e-01, -2.79802829e-01,  2.82918900e-01,  4.77648616e-01,
        8.67762864e-02, -4.74859178e-01,  7.42306113e-02, -1.47349238e-01,
       -5.07345438e-01,  2.15872571e-01,  2.80527025e-01,  1.66475609e-01,
       -2.06533462e-01, -